In [1]:
from qft import *
from qiskit import qasm2
import pandas as pd

In [ ]:
num_q = 20
cir = QFT(num_q)
cir.draw()

In [ ]:
cir.gate_list

In [ ]:
maps = cir.LNN_maps()
maps

In [2]:
from Enola.route import *

In [6]:
route = QuantumRouter(num_q,maps,cir.gate_list,[16,16],"others")
route.run()

In [ ]:
route.movement_list

In [ ]:
route.save_program("test_code_full.json")

In [3]:
for n in range(5,101):
    cir = QFT(n)
    cir.export_to_qasm(f"qft/qft_{n}.qasm",full= True)
    cir.export_to_qasm(f"qft_cz/qft_{n}.qasm",full= True)
for n in range(200,1001,100):
    cir = QFT(n)
    cir.export_to_qasm(f"qft/qft_{n}.qasm",full= True)
    cir.export_to_qasm(f"qft_cz/qft_{n}.qasm",full= True)

QASM file 'qft/qft_5.qasm' has been created.
QASM file 'qft_cz/qft_5.qasm' has been created.
QASM file 'qft/qft_6.qasm' has been created.
QASM file 'qft_cz/qft_6.qasm' has been created.
QASM file 'qft/qft_7.qasm' has been created.
QASM file 'qft_cz/qft_7.qasm' has been created.
QASM file 'qft/qft_8.qasm' has been created.
QASM file 'qft_cz/qft_8.qasm' has been created.
QASM file 'qft/qft_9.qasm' has been created.
QASM file 'qft_cz/qft_9.qasm' has been created.
QASM file 'qft/qft_10.qasm' has been created.
QASM file 'qft_cz/qft_10.qasm' has been created.
QASM file 'qft/qft_11.qasm' has been created.
QASM file 'qft_cz/qft_11.qasm' has been created.
QASM file 'qft/qft_12.qasm' has been created.
QASM file 'qft_cz/qft_12.qasm' has been created.
QASM file 'qft/qft_13.qasm' has been created.
QASM file 'qft_cz/qft_13.qasm' has been created.
QASM file 'qft/qft_14.qasm' has been created.
QASM file 'qft_cz/qft_14.qasm' has been created.
QASM file 'qft/qft_15.qasm' has been created.
QASM file 'qft

In [ ]:
c = -1

In [ ]:
for n in range(5,101):
    cir = QFT(n)
    print(n)
    maps = cir.LNN_maps(-1)
    route = QuantumRouter(cir.num_qubits,maps,cir.gate_list, [cir.num_qubits+2,3],"others")
    route.run()
    route.save_program(f"{'linear_map' if c < 0 else 'fold_map'}/qft_{n}_full_code.json")

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [ ]:
for n in range(200,1001,100):
    cir = QFT(n)
    print(n)
    maps = cir.LNN_maps(-1)
    route = QuantumRouter(cir.num_qubits,maps,cir.gate_list, [cir.num_qubits+2,3],"others")
    route.run()
    route.save_program(f"{'linear_map' if c < 0 else 'fold_map'}/qft_{n}_full_code.json")